In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time
import ast

from sklearn.metrics.pairwise import cosine_similarity

from scipy import sparse

import pickle

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
train_df = pd.read_csv("train_df.csv")
user_df = pd.read_csv('PA_restaurant_user_with_loc.csv')
restaurant_w_train_ave_stars_df=pd.read_csv('restaurant_w_train_ave_stars.csv')

/var/folders/9_/y5d0d5xj7ns0wpcgpkm97z280000gn/T/ipykernel_1475/498966770.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv('PA_restaurant_user_with_loc.csv')


### Popularity Based Model

In [3]:
## Process Opening hour information for further usage

def safe_literal_eval(x):
  if isinstance(x, str):
    try:
      return ast.literal_eval(x)
    except (ValueError, SyntaxError):
      return None
  return None



restaurant_w_train_ave_stars_df['hours'] = restaurant_w_train_ave_stars_df['hours'].apply(safe_literal_eval)
restaurant_w_train_ave_stars_df.head()

,business_id,city,postal_code,categories,yelp_rating,name,RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsDelivery,price_level,...,editorial_summary,good_for_watching_sports,menu_for_children,parking_options,editorial_summary_language,weighted_ave_stars,latitude,longitude,review_count,hours
0,--epgcb7xHGuJ-4PUeSLAw,Willow Grove,19090,"Restaurants, Food, Bagels, Sandwiches, Breakfa...",3.0,Manhattan Bagel,True,True,True,1.0,...,Counter-serve chain specializing in NYC-style ...,False,False,"['freeParkingLot', 'freeStreetParking']",en,2.230112,40.145054,-75.116293,34,"{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', ..."
1,-0FX23yAacC4bbLaGPvyxw,Newtown,18940,"American (Traditional), Restaurants",3.5,The Grey Stone Fine Food and Spirits,True,True,True,3.0,...,NaN,False,True,"['freeParkingLot', 'freeStreetParking', 'valet...",NaN,3.212927,40.256082,-74.916976,155,"{'Tuesday': '12:0-20:0', 'Wednesday': '12:0-20..."
2,-0TffRSXXIlBYVbb5AwfTg,Philadelphia,19107,"Cocktail Bars, Food Delivery Services, Nightli...",4.5,IndeBlue Modern Indian Food & Spirits,True,True,True,2.0,...,Modern & traditional Indian fare plus a tastin...,False,False,"['freeParkingLot', 'freeStreetParking', 'no fr...",en,3.563777,39.948508,-75.161969,1097,"{'Monday': '0:0-0:0', 'Tuesday': '16:0-22:0', ..."
3,-1B9pP_CrRBJYPICE5WbRA,Philadelphia,19107,"Asian Fusion, Restaurants, American (New), Tha...",4.0,Spice 28,True,True,True,2.0,...,"Noodles, fried rice & other classic Asian main...",False,False,[],en,2.885079,39.950352,-75.161583,822,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:30..."
4,-3725FZiIIYdwQtM4MKEIA,Glen Mills,19342,"Pizza, Sandwiches, Chicken Wings, Restaurants",1.5,Domino's Pizza,True,True,True,1.0,...,Delivery/carryout chain offering a wide range ...,False,False,"['freeParkingLot', 'freeStreetParking']",en,1.348674,39.883915,-75.536518,20,"{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'..."


In [4]:
def within_10km(lat, lon, df):
  ## Filter for restaurants that are within 10km away from the estimated user location
  ## distance estimated using latitude and longitude, using roughtly 0.09 as estimation of 10km
  lat_diff = 0.09
  lon_diff = 0.09

  filtered_df = df[((df['latitude'] >= lat - lat_diff) & (df['latitude'] <= lat + lat_diff)) &
                   ((df['longitude'] >= lon - lon_diff) & (df['longitude'] <= lon + lon_diff))]

  return filtered_df

def is_open_now(schedule_dict):
    ## Check if the restaurant is opened at the time of query
    if not schedule_dict:
        return False

    now = datetime.now()
    today = now.strftime('%A')
    current_time = now.time()

    hours = schedule_dict.get(today)
    if not hours or hours == '0:0-0:0':
        return False

    open_str, close_str = hours.split('-')
    try:
        open_hour, open_minute = map(int, open_str.split(':'))
        close_hour, close_minute = map(int, close_str.split(':'))

        open_time = time(open_hour, open_minute)
        close_time = time(close_hour, close_minute)

        return open_time <= current_time <= close_time
    except:
        return False
    
def get_user_loc(user_id, user_df):
  user_loc = user_df[user_df['user_id'] == user_id][["ave_latitude", "ave_longitude"]].values
  return user_loc

def nearest_k_restaurants(lat, lon, restaurant_df, k):
  restaurant_df['distance'] = np.sqrt((restaurant_df['latitude'] - lat)**2 + (restaurant_df['longitude'] - lon)**2)
  return restaurant_df.sort_values(by='distance').head(k)

def review_count_based_recommendation(user_id, user_df, restaurant_df, k, open_now=False):

  user_loc_df = get_user_loc(user_id, user_df)

  if open_now:
    restaurant_df = restaurant_df[restaurant_df['hours'].apply(is_open_now)]


  restaurant_sorted_df = restaurant_df.sort_values(by='review_count', ascending=False)
  high_rated_df = restaurant_sorted_df[restaurant_sorted_df['weighted_ave_stars'] >= 4]

  if len(user_loc_df) > 0:
    user_loc = user_loc_df[0]
    within_10km_df = within_10km(user_loc[0], user_loc[1], high_rated_df)
    if len(within_10km_df) < k:
      within_10km_df = nearest_k_restaurants(user_loc[0], user_loc[1], high_rated_df, 2*k)
  else:
    within_10km_df = high_rated_df

  restaurant_sorted_df = within_10km_df.sort_values(by='review_count', ascending=False)
  high_rated_df = restaurant_sorted_df[restaurant_sorted_df['weighted_ave_stars'] >= 4]
  top_k_list = high_rated_df.head(k)

  return top_k_list['business_id'].tolist()

In [5]:
def get_popularity_based_recommendation(user_id, k):
    recommended_ids = review_count_based_recommendation(user_id, user_df, restaurant_w_train_ave_stars_df, k)
    return pd.DataFrame({
        'business_id': recommended_ids
    })

### Content Based Model

In [6]:
cb_matrix = sparse.load_npz("cb_matrix.npz")
item_df = pd.read_csv("item_df.csv")

text_features = ['categories', 'editorial_summary',
                 'RestaurantsAttire', 'Ambience',
                 'NoiseLevel', 'Music', 'city']

list_features = ['GoodForMeal', 'BusinessParking']

bool_features = [
    'RestaurantsTakeOut', 'RestaurantsDelivery', 'RestaurantsReservations', 'OutdoorSeating',
    'RestaurantsGoodForGroups', 'GoodForKids', 'DogsAllowed', 'dine_in', 'serves_beer', 'serves_wine',
    'serves_cocktails', 'good_for_watching_sports', 'serves_coffee', 'menu_for_children',
    'BusinessAcceptsCreditCards', 'HasTV', 'BikeParking', 'Caters', 'RestaurantsTableService',
    'WheelchairAccessible', 'HappyHour', 'BusinessAcceptsBitcoin'
]

num_features = [
    'yelp_rating', 'number_of_photos', 'dist_highway',
    'rural_urban_continuum_code_2023', 'adjusted_gross_income',
    'unemployment_rate_2023', 'user_rating_count',
    'google_rating', 'price_level'
]

features = ['business_id', 'name', 'description'] + \
    text_features + list_features + bool_features + num_features

def get_content_based_recommendation(user_id, k):
    past_restaurants = train_df.loc[(train_df['user_id'] == user_id)]
    restaurant_indices = []
    aligned_ratings = []

    for _, row in past_restaurants.iterrows():
        business_id = row['business_id']
        rating = row['stars']
        idx = item_df[item_df['business_id'] == business_id].index
        if len(idx) > 0:
            restaurant_indices.append(idx[0])
            aligned_ratings.append(rating)

    vectors = cb_matrix[restaurant_indices].toarray()
    weights = np.array(aligned_ratings) - 3  # center ratings at neutral (3)
    weights = weights.reshape(-1, 1)

    weighted_embedding = (vectors * weights).sum(axis=0)
    query_embedding = weighted_embedding.reshape(1, -1)

    similarity_scores = cosine_similarity(
        query_embedding, cb_matrix).flatten()

    similarity_scores[similarity_scores >= 0.99] = 0

    top_k_indices = np.argsort(similarity_scores.flatten())[::-1][:k]
    top_k_scores = similarity_scores.flatten()[top_k_indices]
    recommended_ids = item_df.iloc[top_k_indices]['business_id'].values

    recommended_df = pd.DataFrame({
        'business_id': recommended_ids,
        'similarity_score': top_k_scores
    })

    recommended_df['similarity_score'] = recommended_df['similarity_score'].round(
        3)

    return recommended_df.reset_index(drop=True)

### Collaborative Filtering Model

In [7]:
cf_matrix = sparse.load_npz("cf_matrix.npz")

with open('user_encoder.pkl', 'rb') as f:
    user_encoder = pickle.load(f)

with open('item_encoder.pkl', 'rb') as f:
    item_encoder = pickle.load(f)

with open('user_decoder.pkl', 'rb') as f:
    user_decoder = pickle.load(f)

users_df = pd.read_csv("users_df.csv")


def get_collaborative_filtering_recommendation(user_id, k, top_k_indices, similarity_scores):
    # user_idx = user_encoder[user_id]
    # query_vector = cf_matrix[user_idx].reshape(1, -1)

    # similarity_scores = cosine_similarity(
    #     query_vector, cf_matrix).flatten()
    # similarity_scores[user_idx] = 0  # remove self-similarity

    # # Get top-k similar users
    # top_k_indices = np.argsort(similarity_scores)[::-1][:k]
    top_k_user_ids = [users_df["user_id"][i] for i in top_k_indices]
    top_k_similarities = similarity_scores[top_k_indices]

    # Get ratings from these similar users
    sim_user_ratings = train_df[train_df['user_id'].isin(
        top_k_user_ids)].copy()
    sim_score_dict = dict(zip(top_k_user_ids, top_k_similarities))
    sim_user_ratings['weight'] = sim_user_ratings['user_id'].map(
        sim_score_dict)
    sim_user_ratings['weighted_ratings'] = sim_user_ratings['stars'] * \
        sim_user_ratings['weight']

    # Aggregate predictions
    agg_df = sim_user_ratings.groupby('business_id').agg(
        predicted_score=('weighted_ratings', 'sum'),
        total_weight=('weight', 'sum')
    )
    agg_df['predicted_score'] = agg_df['predicted_score'] / \
        agg_df['total_weight']

    # Select top-k
    top_recs = agg_df.sort_values(
        by='predicted_score', ascending=False).head(k).reset_index()

    return top_recs[['business_id', 'predicted_score']]

### Matrix Factorization Model

In [9]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, user_feat_dim, item_feat_dim, embedding_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, embedding_dim)
        self.item_emb = nn.Embedding(n_items, embedding_dim)

        # Side feature projection layers
        self.user_feat_proj = nn.Linear(user_feat_dim, embedding_dim)
        self.item_feat_proj = nn.Linear(item_feat_dim, embedding_dim)

        # Bias terms
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)
        self.global_bias = nn.Parameter(torch.tensor([0.0]))

        self.dropout = nn.Dropout(0.3)

        # Initialization
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)
        nn.init.xavier_uniform_(self.user_feat_proj.weight)
        nn.init.xavier_uniform_(self.item_feat_proj.weight)

    def forward(self, users, items, user_features, item_features):
        # User latent + side
        u_id = self.dropout(self.user_emb(users))
        u_feat = self.user_feat_proj(user_features[users])
        u = u_id + u_feat

        # Item latent + side
        i_id = self.dropout(self.item_emb(items))
        i_feat = self.item_feat_proj(item_features[items])
        i = i_id + i_feat

        # Dot product + bias
        dot = (u * i).sum(1, keepdim=True)
        bias = self.user_bias(users) + self.item_bias(items) + self.global_bias

        return (dot + bias).squeeze(1)  
    

num_users = 269461
num_items = 8069
user_features_tensor = torch.load('user_features_tensor.pt')
item_features_tensor = torch.load('item_features_tensor.pt')
all_item_indices=list(range(item_features_tensor.shape[0]))
mf_model = MF(num_users, num_items, 3, 386)
mf_model.load_state_dict(torch.load('MF_model.pth'))
mf_model.eval()

/var/folders/9_/y5d0d5xj7ns0wpcgpkm97z280000gn/T/ipykernel_1475/2733985164.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  user_features_tensor = torch.load('user_featu

MF(
  (user_emb): Embedding(269461, 32)
  (item_emb): Embedding(8069, 32)
  (user_feat_proj): Linear(in_features=3, out_features=32, bias=True)
  (item_feat_proj): Linear(in_features=386, out_features=32, bias=True)
  (user_bias): Embedding(269461, 1)
  (item_bias): Embedding(8069, 1)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [10]:
with open('user_mf_encoder.pkl', 'rb') as f:
    user_le = pickle.load(f)

with open('item_mf_encoder.pkl', 'rb') as f:
    item_le = pickle.load(f)

@torch.no_grad()
def get_mf_recommendation(model, user_idx, all_item_indices, user_features_tensor, item_features_tensor, device='cpu', top_k=10):
    model.eval()

    user_tensor = torch.tensor([user_idx] * len(all_item_indices), dtype=torch.long).to(device)
    item_tensor = torch.tensor(all_item_indices, dtype=torch.long).to(device)

    user_features_tensor = user_features_tensor.to(device)
    item_features_tensor = item_features_tensor.to(device)

    # Predict scores for all items for this user
    scores = model(user_tensor, item_tensor, user_features_tensor, item_features_tensor)
    scores = torch.clamp(scores, 1, 5)
    scores = scores.cpu().numpy()

    # Rank items by score
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    top_items = [(all_item_indices[i], scores[i]) for i in ranked_indices]

    item_indices = [item_idx for item_idx, _ in top_items]
    original_business_ids = item_le.inverse_transform(item_indices)  
    top_recs = pd.DataFrame({'business_id': original_business_ids})
    return top_recs

/Users/liusiyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Diverse Selection

In [11]:
tfidf_scores = sparse.load_npz("ds_matrix.npz")

In [12]:
def cos_sim(vector_a, vector_b):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Calculate the Euclidean norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Calculate cosine similarity using the dot product and vector norms
    similarity = dot_product / (norm_a * norm_b)
    return similarity

def pairwise_cos_sim(vectors):
    num_vectors = vectors.shape[0]
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            # Calculate cosine similarity for each pair of vectors
            similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

    return similarity_matrix

def pick_diverse_items(similarity_matrix, n_items=10):
    N = similarity_matrix.shape[0]
    selected = []

    # Step 1: Start with the item most similar to all others (highest avg sim)
    avg_sims = similarity_matrix.mean(axis=1)
    first = np.argmax(avg_sims)
    selected.append(first)

    # Step 2: Iteratively pick items least similar to the current selection
    while len(selected) < n_items:
        remaining = [i for i in range(N) if i not in selected]
        dissimilarities = []

        for i in remaining:
            avg_sim_to_selected = similarity_matrix[i, selected].mean()
            dissimilarities.append((i, avg_sim_to_selected))

        # Pick the item with the lowest similarity to selected set
        next_item = min(dissimilarities, key=lambda x: x[1])[0]
        selected.append(next_item)

    return selected

In [13]:
def diverse_select(recom_list):
    matching_indices = restaurant_w_train_ave_stars_df.index[restaurant_w_train_ave_stars_df['business_id'].isin(recom_list)].tolist()
    matching_tfidf_scores = tfidf_scores[matching_indices]
    tfidf_cos_sim = pairwise_cos_sim(matching_tfidf_scores.toarray())
    selected_indices = pick_diverse_items(tfidf_cos_sim, n_items=10)
    original_indices = [matching_indices[i] for i in selected_indices]
    selected_values = restaurant_w_train_ave_stars_df.iloc[original_indices]['business_id']

    return pd.DataFrame({'business_id': selected_values})


### Combine All Models

In [14]:
def get_recommendation(user_id, k, visualize=True, diverse=False):
    # If user is not found aka "new user"
    if user_id not in user_encoder: 
        recommendations = get_popularity_based_recommendation(user_id, k)
        recommendations = recommendations.merge(
            item_df[features], on='business_id', how='left')
        return recommendations
    
    # For "existing user", find top-k similar users
    user_idx = user_encoder[user_id]
    query_vector = cf_matrix[user_idx].reshape(1, -1)
    similarity_scores = cosine_similarity(
        query_vector, cf_matrix).flatten()
    similarity_scores[user_idx] = 0  # remove self-similarity
    top_k_indices = np.argsort(similarity_scores)[::-1][:k]

    

    # If there are similar users
    if top_k_indices.size > 0:
        cb_rec = get_content_based_recommendation(user_id, k)[["business_id"]]
        cf_rec = get_collaborative_filtering_recommendation(user_id, k, top_k_indices, similarity_scores)[
            ["business_id"]]
        mf_rec = get_mf_recommendation(mf_model, user_idx, all_item_indices, user_features_tensor, item_features_tensor, device='cpu', top_k=k)
        cb_rec["rank"] = cb_rec.index + 1
        cb_rec["weight"] = 0.4
        cf_rec["rank"] = cf_rec.index + 1
        cf_rec["weight"] = 0.4
        mf_rec["rank"] = mf_rec.index + 1
        mf_rec["weight"] = 0.2
        recommendations = pd.concat([mf_rec,cb_rec, cf_rec])
        
    # If there are no similar users    
    else:
        cb_rec = get_content_based_recommendation(user_id, k)[["business_id"]]
        mf_rec = get_mf_recommendation(mf_model, user_idx, all_item_indices, user_features_tensor, item_features_tensor, device='cpu', top_k=k)
        cb_rec["rank"] = cb_rec.index + 1
        cb_rec["weight"] = 0.6
        mf_rec["rank"] = mf_rec.index + 1
        mf_rec["weight"] = 0.4
        recommendations = pd.concat([mf_rec, cb_rec])
    
    # Tidy up recommendations
    if diverse:
        recom_list = recommendations['business_id'].unique()
        diverse_list = diverse_select(recom_list)
        recommendations = diverse_list.merge(item_df[features], on='business_id', how='left')

    else:
        recommendations['weighted_rank'] = recommendations['rank'] * recommendations['weight']
        recommendations = recommendations.groupby("business_id")["weighted_rank"].sum().reset_index().sort_values("weighted_rank").head(k)
        recommendations = recommendations.merge(
            item_df[features], on='business_id', how='left')

    return recommendations

### Get Recommendations

In [16]:
get_recommendation('mh_-eMZ6K5RLWhZyISBhwA', 10)

,business_id,weighted_rank,name,description,categories,editorial_summary,RestaurantsAttire,Ambience,NoiseLevel,Music,...,BusinessAcceptsBitcoin,yelp_rating,number_of_photos,dist_highway,rural_urban_continuum_code_2023,adjusted_gross_income,unemployment_rate_2023,user_rating_count,google_rating,price_level
0,NvOMeOZp6SC702ZqOLM4Cg,0.2,Nudy's Café - West Chester,restaurant breakfast brunch salad burger casua...,Restaurants Breakfast & Brunch Salad Burgers,Casual breakfast & lunch spot with a large men...,casual,classy casual,average,NaN,...,False,3.5,10.0,3.431273,1.0,3458385.0,2.900000,726.0,4.3,2.0
1,u50hTvPV_W_Hx625ytvLYw,0.4,Hatboro Pizza,salad food delivery service italian event plan...,Salad Food Delivery Services Italian Event ...,Quaint family-run pizzeria serves traditional ...,casual,casual,average,NaN,...,NaN,3.5,10.0,1.633556,1.0,877828.0,2.900000,357.0,4.4,1.0
2,_-8TAMmIbDGkZCXAqSkFMg,0.4,Slack's Hoagie Shack,restaurant caterer pizza event planning servic...,Restaurants Caterers Pizza Event Planning &...,Chain sandwich specialist turning out hoagies ...,casual,casual,quiet,NaN,...,NaN,3.5,10.0,1.554484,1.0,1237238.0,3.200000,294.0,4.2,1.0
3,WL-0PLW5IzdnyUHGmiOrgQ,0.4,Keswick Tavern,bar greek restaurant nightlife beer wine spiri...,Bars Greek Restaurants Nightlife Beer Win...,Contemporary tavern with multiple TVs for spor...,casual,classy casual,average,live,...,NaN,3.5,10.0,0.687538,1.0,1739529.0,2.800000,656.0,4.2,2.0
4,OTwwIvLkrS0zc8f1lNRstw,0.6,Dasiwa,sushi bar restaurant intimate bistro providing...,Sushi Bars Restaurants,Intimate bistro providing sushi & other Japane...,casual,classy casual,quiet,NaN,...,False,4.5,10.0,0.930093,1.0,1650111.0,4.200000,222.0,4.5,2.0
5,8n_BlTxfALO08FWdb6Tnbg,0.8,Pholosophy,restaurant vietnamese restaurant vietnamese ca...,Restaurants Vietnamese,Restaurants Vietnamese,NaN,casual,average,NaN,...,False,4.0,10.0,1.757572,1.0,1143777.0,2.800000,186.0,4.5,1.0
6,xdG_RW_QsAGJbXp46M_V8w,0.8,MOD Pizza,pizza restaurant fast food chain restaurant kn...,Pizza Restaurants Fast Food,Counter-serve industrial-chic chain restauran...,casual,casual,average,NaN,...,False,4.0,10.0,2.099933,1.0,3836907.0,2.866667,599.0,4.3,1.0
7,rQ5xsCMH6Xdk4NnmdeA3Eg,0.8,Pineville Tavern,nightlife restaurant american traditional bar ...,Nightlife Restaurants American (Traditional)...,From-scratch American fare including steaks & ...,casual,casual,average,NaN,...,NaN,3.0,10.0,10.379030,1.0,504868.0,3.000000,1532.0,4.1,2.0
8,aCDY7vXYMs54EbYuQScsnQ,1.0,39 North Taproom & Grill,american traditional sport bar restaurant bar ...,American (Traditional) Sports Bars Restauran...,American (Traditional) Sports Bars Restauran...,NaN,NaN,NaN,live,...,NaN,4.5,10.0,3.307320,1.0,358646.0,3.200000,366.0,4.4,2.0
9,-rwfGlw6T5czqQO4uZGWYw,1.2,Moonlight Diner,breakfast brunch diner mediterranean restauran...,Breakfast & Brunch Diners Mediterranean Res...,Relaxed neighborhood nook providing classic A...,casual,casual,average,NaN,...,NaN,4.0,10.0,0.687538,1.0,1739529.0,2.800000,736.0,4.3,1.0


In [17]:
get_recommendation('mh_-eMZ6K5RLWhZyISBhwA', 10, diverse=True)

,business_id,name,description,categories,editorial_summary,RestaurantsAttire,Ambience,NoiseLevel,Music,city,...,BusinessAcceptsBitcoin,yelp_rating,number_of_photos,dist_highway,rural_urban_continuum_code_2023,adjusted_gross_income,unemployment_rate_2023,user_rating_count,google_rating,price_level
0,xdG_RW_QsAGJbXp46M_V8w,MOD Pizza,pizza restaurant fast food chain restaurant kn...,Pizza Restaurants Fast Food,Counter-serve industrial-chic chain restauran...,casual,casual,average,NaN,Wayne,...,False,4.0,10.0,2.099933,1.0,3836907.0,2.866667,599.0,4.3,1.0
1,aCDY7vXYMs54EbYuQScsnQ,39 North Taproom & Grill,american traditional sport bar restaurant bar ...,American (Traditional) Sports Bars Restauran...,American (Traditional) Sports Bars Restauran...,NaN,NaN,NaN,live,Glenolden,...,NaN,4.5,10.0,3.307320,1.0,358646.0,3.200000,366.0,4.4,2.0
2,8n_BlTxfALO08FWdb6Tnbg,Pholosophy,restaurant vietnamese restaurant vietnamese ca...,Restaurants Vietnamese,Restaurants Vietnamese,NaN,casual,average,NaN,Narberth,...,False,4.0,10.0,1.757572,1.0,1143777.0,2.800000,186.0,4.5,1.0
3,YjLMWlHoBJHtYMLdFXfvVg,El Limon - Ambler,mexican restaurant salad sandwich familiar mex...,Mexican Restaurants Salad Sandwiches,Familiar Mexican grub including tostadas tor...,casual,casual,average,NaN,Ambler,...,NaN,4.0,10.0,0.029601,1.0,2886349.0,2.800000,1465.0,4.5,1.0
4,gpYBhnTk4KzvvH83TsZiQg,Humpty's Dumplings,food delivery service american new restaurant ...,Food Delivery Services American (New) Restau...,No-frills counter-serve dishing up sweet & sav...,casual,hipster casual,average,NaN,Glenside,...,NaN,4.0,10.0,0.687538,1.0,1739529.0,2.800000,606.0,4.4,1.0
5,OTwwIvLkrS0zc8f1lNRstw,Dasiwa,sushi bar restaurant intimate bistro providing...,Sushi Bars Restaurants,Intimate bistro providing sushi & other Japane...,casual,classy casual,quiet,NaN,Philadelphia,...,False,4.5,10.0,0.930093,1.0,1650111.0,4.200000,222.0,4.5,2.0
6,NQ01WqVX0tojNHKn-0sFww,Tir na nOg Irish Pub,irish pub nightlife american new bar irish pub...,Irish Pub Nightlife American (New) Bars Ir...,Draft beers plus Irish & American pub fare fo...,casual,casual,loud,live,Philadelphia,...,NaN,3.5,10.0,0.669666,1.0,3058841.0,4.200000,1182.0,4.2,2.0
7,YHvJyyklyzsQhCnbJu7pFA,Nino's Pizzarama - Willow Grove,pizza italian restaurant italian eatery dishin...,Pizza Italian Restaurants,Family-run Italian eatery dishing up whole & b...,casual,casual,average,NaN,Willow Grove,...,NaN,4.0,10.0,1.322643,1.0,713838.0,2.800000,597.0,4.3,1.0
8,HNREwZWJqeapl4-uRhaKNg,PrimoHoagies,sandwich restaurant italian sandwich chain kno...,Sandwiches Restaurants Italian,Philly-born counter-serve sandwich chain known...,casual,NaN,average,NaN,Hatboro,...,NaN,3.5,10.0,1.633556,1.0,877828.0,2.900000,300.0,4.4,1.0
9,X-lGF6aL8RIO-3ROlxcZYA,California Tortilla,vegetarian restaurant mexican chain burrito qu...,Vegetarian Restaurants Tex-Mex Mexican,Counter-serve chain for made-to-order burritos...,casual,casual,average,NaN,Royersford,...,NaN,4.0,10.0,0.527104,1.0,1305569.0,2.800000,973.0,4.2,1.0
